# Problem description and notation

We have $N$ hard drives we want to use for backup. Each one has capacity $C_i$ for $i=1 \ldots N$. For each file to be backed up we want to store $R$ replicas and we want the hard drives to be more or less used according to their capacity in such a way that, after some time being used, all of them are, for example, at 60% of their capacity, instead of one being fully filled and others empty.

Let's call $C$ the total capacity of the hard drives:
$$C=\sum_{i=1}^N C_i$$

Let's call also $c_i$ the fraction of the total capacity that hard drive $i$ provides:
$$c_i = \frac{C_i}{C}$$

One simple algorithm if we want to just make one replica is to randomly select one hard drive each time we want to store a replica, with probability proportional to the hard drive capacity. Let's call $S=i$ the event that consists of selecting hard drive $i$. Then:

$$ P(S=i) = c_i $$

In the following sections we discuss the case where $R > 1$. We want to continue filling the hard drives proportionally to their capacity, but with the constraint of not storing two replicas of the same file in the same hard drive.

# Two replicas (R=2)

Before discussing the general case it's better to analyse this particular case. Now, each time a new file is to be replicated, we must select two hard drives. Let's call the action of selecting the first $S_1$ and the second $S_2$. We want to calculate the probabilities:

$$ P(S_1=i, S_2=j) $$

Let's call $S=i$ the event of selecting hard drive $i$ as the first or the second hard drive. Since we cannot select the same hard drive for the two replicas the events are disjoint and we have that:

$$ P(S=i) = P(S_1 = i \cup S_2 = i) = P(S_1 = i) + P(S_2 = i) $$

We want this probability to be proportional as before to the hard drive capacity. It must be then that:

$$ P(S=i) = kc_i$$

For some constant $k$. We can determine the value of this constant since:

$$ \sum_{i=1}^N P(S=i) = \sum_{i=1}^n \left[P(S_1 = i) + P(S_2 = i)\right] = \sum_{i=1}^N P(S_1=i) + \sum_{i=1}^NP(S_2=i) = 1 + 1 = 2 $$

Or more in general:

$$ \sum_{i=1}^N P(S=i) = R $$

And so we have that $k=R$:

$$ P(S = i) = Rc_i $$

Let's write the joint distribution $P(S_1=i, S_2=j)$ using the chain rule:

$$ P(S_1=i, S_2=j) = P(S_1=i)P(S_2=j|S_1=i) $$

Let's suppose that we choose the two hard drives independently and if we have made an incorrect selection we discard the selection and select again. More exactly, we select the first hard drive we probability $p_1^i$ and call this event $T_1$ and the second hard drive we probability $p_2^j$ and call this event $T_2$:

$$ P(T_1=i, T_2=j) = p_1^ip_2^j $$

The probability distribution of the accepted hard drives selections would be:

$$ P(S_1 = i, S_2=j) = P(T_1=i, T_2=j | i \neq j) = P(T_1=i, T_2=j) / P(j \neq i) = p_1^i \frac{p_2^j}{1 - p_2^i} $$

We can compute the marginal probabilities easily:

$$ 
\begin{split}
P(S_1 = i) &=& \sum_{j \neq i} P(S_1=i, S_2=j) &=& p_1^i \\
P(S_2 = i) &=& \sum_{j \neq j} P(S_1=j, S_2=i) &=& p_2^i\sum_{j \neq i}\frac{p_1^j}{1 - p_2^j}
\end{split}
$$

And the event of selecting hard drive $i$:

$$ P(S = i) = p_1^i + p_2^i\sum_{j \neq i}\frac{p_1^j}{1 - p_2^j} = Rc_i$$

From these equations we have the following iterative equations that adjusts the probabilities of the second hard drive given the probabilities of the first:

$$ 
\begin{split}
\left.p_2^i\right|_0 &=& Rc_i \\
\left.p_2^i\right|_{k+1} &=& \frac{Rc_i - p_1^i}{
        \sum_{j \neq i}\frac{p_1^j}{1 - \left.p_2^j\right|_k}} 
\end{split}
$$

If we want that the first replica is equally distributed on all hard drives then it must be that:

$$ p_1^i = \frac{1}{N} $$

Notice that the above equations don't have solution if for some $i$ we have that:

$$ c_i < \frac{1}{RN} $$

Or equivalently:

$$ C_i < \frac{1}{R}\frac{C}{N} $$

This happens, for the case we are considering here of $R=2$, if some hard disk has less than half than the average capacity. In this case we can try to distribute it as uniformly as possible. Let's call $L$ the set of hard drives that are too small:

$$ L = \left\{i \mid c_i < \frac{1}{RN} \right\} $$

For the hard drives in this set ($i \in L$) we define:

$$ p_1^i = Rc_i $$

And for the hard drives not in this set ($i \notin L$):

$$ p_1^i = \frac{1}{N - |L|}\left( 1 - \sum_{j \in L}p_1^j \right) $$

In [1]:
import warnings
import numpy as np


def init_p_1(c, R):
    N = len(c)
    L = c < 1/(R*N)
    M = np.logical_not(L)    
    p_1 = np.empty_like(c)
    p_1[L] = R*c[L]
    p_1[M] = 1/np.sum(M)*(1 - np.sum(p_1[L]))
    return p_1
    
def update_p_2(p_1, p_2, c, R=2):
    u = p_1 / (1 - p_2)
    v = np.sum(u)
    return (R*c - p_1) / (v - u)


def _probabilities(c, update_func, R, tol=1e-3, max_iter=1000, verbose=0):
    c = c/np.sum(c)
    
    # Probabilities of selecting first hard drive
    p_1 = init_p_1(c, R)

    # Probabilities of selecting second hard drive
    p_2_old = np.copy(c)
    eps = np.inf
    it = 0
    while eps > tol:                
        p_2_new = update_func(p_1, p_2_old, c, R)
        p_2_new /= np.sum(p_2_new)  # numerical corrections
        eps = np.max(np.abs(p_2_new - p_2_old))
        it += 1
        if eps < tol:
            break
        if it >= max_iter:
            if tol > 0:
                warnings.warn('Maximum iterations reached')
            break
        if verbose > 0:
            print(it, eps)
            if verbose > 1:
                for pi in p_2_new:
                    print('    ', pi)
        p_2_old, p_2_new = p_2_new, p_2_old

    return p_1, p_2_new


def probabilities(c):
    return _probabilities(c, update_p_2, 2)

To test the algorithm we make some simulations. The following function builds a selector which is simply a function with no arguments that selects two hard drives:

In [2]:
def build_selector(p_1, p_2, R=2):
    def selector():
        i = np.random.choice(len(p_1), p=p_1)
        selection = [i]
        p = np.copy(p_2)
        for r in range(1, R):
            p[i] = 0
            p /= np.sum(p)
            i = np.random.choice(len(p), p=p)
            selection.append(i)        
        return selection
    return selector

The simulation code simply runs a selector repeatidly and for each hard drive marks with 0 if it's not selected, 1 if it's selected for the first replica and 2 if it's selected for the second.

In [3]:
def run_sim(N, selector, n=100000):
    results = np.zeros((n, N), dtype=int)
    for it in range(n):
        selected = selector()
        for r, i in enumerate(selected):
            results[it, i] = r + 1
    return results


def report(sim, expected, actual):
    print('Expected vs actual use ({0} samples)'.format(sim.shape[0]))
    for i in range(len(expected)):
        print(' disk {0}: {1:.2e} {2:.2e}'.format(i, expected[i], actual[i]))

In [4]:
cap_1 = np.array([10, 8, 6, 6])  # avg cap: 7.5
cap_2 = np.array([10, 6, 6, 2])  # avg cap: 6

print('Simulation 1')
sim_1 = run_sim(len(cap_1), build_selector(*probabilities(cap_1)))
print('First replica on each hard drive')
report(sim_1, np.repeat(0.25, 4), np.mean(sim_1 == 1, axis=0))
print('All replicas on each hard drive')
report(sim_1, cap_1/np.sum(cap_1), 0.5*np.mean(sim_1 > 0, axis=0))
print()

print('Simulation 2')
sim_2 = run_sim(len(cap_2), build_selector(*probabilities(cap_2)))
print('First replica on each hard drive (could agree just by chance)')
report(sim_2, np.repeat(0.25, 4), np.mean(sim_2 == 1, axis=0))
print('All replicas on each hard drive')
report(sim_2, cap_2/np.sum(cap_2), 0.5*np.mean(sim_2 > 0, axis=0))

Simulation 1
First replica on each hard drive
Expected vs actual use (100000 samples)
 disk 0: 2.50e-01 2.50e-01
 disk 1: 2.50e-01 2.51e-01
 disk 2: 2.50e-01 2.50e-01
 disk 3: 2.50e-01 2.49e-01
All replicas on each hard drive
Expected vs actual use (100000 samples)
 disk 0: 3.33e-01 3.33e-01
 disk 1: 2.67e-01 2.68e-01
 disk 2: 2.00e-01 2.00e-01
 disk 3: 2.00e-01 1.99e-01

Simulation 2
First replica on each hard drive (could agree just by chance)
Expected vs actual use (100000 samples)
 disk 0: 2.50e-01 2.79e-01
 disk 1: 2.50e-01 2.78e-01
 disk 2: 2.50e-01 2.77e-01
 disk 3: 2.50e-01 1.67e-01
All replicas on each hard drive
Expected vs actual use (100000 samples)
 disk 0: 4.17e-01 4.17e-01
 disk 1: 2.50e-01 2.50e-01
 disk 2: 2.50e-01 2.50e-01
 disk 3: 8.33e-02 8.34e-02


# More than two replicas ($R > 2$)

If $R=3$ then the joint probability would be:

$$ P(S_1=i, S_2=j, S_3=k) = p_1^i\frac{p_2^j}{1 - p_2^i}\frac{p_3^k}{1 - p_3^i - p_3^j} $$

And the probability of selecting hard drive $i$:

$$ P(S = i) = p_1^i + p_2^i\sum_{j \neq i}\frac{p_1^j}{1 - p_2^j} + p_3^i\sum_{j \neq i}\frac{p_1^j}{1 - p_2^j}\sum_{k\neq j,i}\frac{p_2^k}{1 - p_3^j - p_3^k} $$

The problem is that as $N$ and $R$ grow not only the formula becomes more complicated but also the complexity of evaluating the marginal probabilities grow as $N^{R-1}$. 

_TODO_